# ELEMENT LEVEL MODEL TRAINING

This notebook contains the necessary code to train a YOLOv8 model for detecting the individual GUI elements in a desktop
screenshot given a raw dataset.

### CONFIG VARS

In [7]:
from utils import *
import cv2
from shapely.geometry import Polygon, Point, GeometryCollection
from ultralytics import YOLO

In [8]:
DATASET_PATH="../Datasets/Dataset_ElementLevel"
PREPROCESSED_PATH="../Preprocessed_Datasets/ElementLevel"
AUGMENTED_PATH="../Augmented_Datasets/ElementLevel"
AUGMENTED_PATH_TRAIN_EX="../Augmented_Datasets/ElementLevel_train" # Data exclusively for training, not validating
YOLO_PATH="../YOLO_Datasets/ElementLevel"
YOLO_PATH_TRAIN_EX="../YOLO_Datasets/ElementLevel_train" # Data exclusively for training, not validating

### AUGMENTATION

For this model we will apply the following augmentation techniques:
- Hue transformations (-100º to +100º)
- Contrast inversion (To simulate dark and light modes)
- Tiling ( Slicing Aided Hyper Inference )

We will first apply tiling and then the other 2, but without applying all 3 to any image. This is because tiling is 
an essential step in our pipeling due to the use of SAHI, but the other 2 should not be combined as to not deviate
too much from what the model will recieve as an input

In [10]:
create_slices(DATASET_PATH, AUGMENTED_PATH, 3, 3, 0.2, 0.2)

We apply resize now and not before because we want all images, including tiles, to keep the image size that the model will take

In [11]:
resize_dataset_images(AUGMENTED_PATH, AUGMENTED_PATH, 640, 360)

In [12]:
hue_augmentation(AUGMENTED_PATH, AUGMENTED_PATH_TRAIN_EX, 0.15, 100)

In [13]:
contrast_inversion_augmentation(AUGMENTED_PATH, AUGMENTED_PATH_TRAIN_EX, 0.15)

In [14]:
# Add the augmented data to the train exclusively folder
for file in os.listdir(AUGMENTED_PATH):
    shutil.copy(os.path.join(AUGMENTED_PATH, file), AUGMENTED_PATH_TRAIN_EX)

### FORMAT CONVERSION

Up to now, we have treated with labelme format datasets, but we need to convert it to YOLOv8 format if we want to train
a model, which has the following format:

```
YOLOv8_Dataset/
├── data.yaml
├── train/
│   ├── images/
│   │   ├── img1.jpg
│   │   ├── img2.jpg
│   │   └── ...
│   ├── labels/
│   │   ├── img1.txt
│   │   ├── img2.txt
│   │   └── ...
├── valid/
│   ├── images/
│   │   ├── img1.jpg
│   │   ├── img2.jpg
│   │   └── ...
│   ├── labels/
│   │   ├── img1.txt
│   │   ├── img2.txt
│   │   └── ...
└── test/ (OPTIONAL)
    ├── images/
    │   ├── img1.jpg
    │   ├── img2.jpg
    │   └── ...
    └── labels/
        ├── img1.txt
        ├── img2.txt
        └── ...
```

The format of the data.yml file is:
```
path: <path_to_dataset_root_dit>
train: <path_to_train_images>
val: <path_to_validation_images>
test: <path_to_test_images> (OPTIONAL)

nc: <number_of_classes>
names: ['class1', 'class2', 'class3', ...]
```

The labels for Instance segmentation have the following format for each annotation:
```
<class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
```

In [15]:
labelme_to_yolo(AUGMENTED_PATH_TRAIN_EX, YOLO_PATH_TRAIN_EX, 0.7,[
                    "WebIcon",
                    "Icon",
                    "Switch",
                    "BtnSq",
                    "BtnPill",
                    "BtnCirc",
                    "CheckboxChecked",
                    "CheckboxUnchecked",
                    "RadiobtnSelected",
                    "RadiobtnUnselected",
                    "TextInput",
                    "Dropdown",
                    "Link",
                ], "seg")

### TRAIN

We will perform fine-tuning over the mobile-sam model using the hyperparameter tuning provided by Ultralytics to get the
best results we can. Since this is a non-standard dataset in terms of object features it is not clear what are the values
we should use.

We will also configure the training to not do any augmentation over the train set

In [1]:
from ultralytics import YOLO
# Initialize the YOLO model
model = YOLO("yolov8n-seg")

In [2]:
import torch
# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using {} device'.format(device))

Using cuda device


In [2]:
# Tune hyperparameters on dataset for 30 epochs
model.tune(data="../YOLO_Datasets/ElementLevel_train/data.yaml", workers=1, epochs=30, iterations=10, optimizer='AdamW', plots=False, save=True, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, translate=0.0, scale=0.0, fliplr=0.0, mosaic=0.0)

Tuner: Initialized Tuner instance with 'tune_dir=runs\segment\tune2'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.0, 'hsv_s': 0.0, 'hsv_v': 0.0, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.0, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.0, 'mosaic': 0.0, 'mixup': 0.0, 'copy_paste': 0.0}
Saved runs\segment\tune2\tune_scatter_plots.png
Saved runs\segment\tune2\tune_fitness.png

Tuner: 1/10 iterations complete  (5651.61s)
Tuner: Results saved to runs\segment\tune2
Tuner: Best fitness=0.81092 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.65464, 'metrics/recall(B)': 0.5716, 'metrics/mAP50(B)': 0.62608, 'metrics/mAP50-95(B)': 0.47866, 'metrics/precision(M)': 0.62258, 'metrics/recall(M

In [3]:
model = YOLO("runs/segment/tune2/weights/best.pt")

In [11]:
metrics.seg.map    # map50-95
metrics.seg.map50  # map50
metrics.seg.map75  # map75
metrics.seg.maps   # a list contains map50-95 of each category

array([    0.19274,     0.32869,     0.40419,     0.39927,       0.561,     0.28032,     0.10029,     0.16814,     0.26469,     0.19651,     0.35886,     0.31838,     0.15703])

In [12]:
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

array([    0.44217,     0.57988,      0.5628,     0.66923,     0.72803,     0.39053,     0.33478,     0.53026,     0.41717,     0.49925,     0.55777,     0.43547,      0.2592])